In [ ]:
from fn_cfg import *
import params as cfg
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import joblib

In [ ]:
#dataset = pd.read_csv('/Users/joshuaighalo/Documents/BrainNet/Projects/Workspace/results/laurel place/ml_dataset/eye_status_20s.csv')
dataset = pd.read_csv('/Users/joshuaighalo/Documents/BrainNet/Projects/Workspace/results/laurel place/ml_dataset/eye_status_10s.csv')
#dataset = pd.read_csv('/Users/joshuaighalo/Documents/BrainNet/Projects/Workspace/results/laurel place/ml_dataset/eye_status_30s.csv')
X = dataset.drop('eye_status', 1)
y = dataset['eye_status'].array

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
pca = PCA()
dfx_pca = pca.fit(X_scaled)
explained_variance = pca.explained_variance_ratio_
idx = np.where(explained_variance > 0.05)[0]
print(X.columns.values[idx])
X_scaled = X_scaled[:,idx]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33)

In [6]:
#SVM
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
rf_params = {
    'C': stats.uniform(0,50),
    "kernel":['linear','poly','rbf','sigmoid']
}
n_iter_search=20
clf = SVC(gamma='scale')
Random = RandomizedSearchCV(clf, param_distributions=rf_params,n_iter=n_iter_search,cv=3,scoring='accuracy')
Random.fit(X_train, y_train)
print(Random.best_estimator_)
print("Accuracy:"+ str(Random.best_score_))

In [ ]:
Random.best_estimator_.fit(X_train, y_train)

In [ ]:
# save the model to disk
filename = 'best_SVC_model.sav'
joblib.dump(Random.best_estimator_, filename)

# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test, y_test)

In [ ]:
#y_pred = Random.best_estimator_.predict(X_train)
y_pred = loaded_model.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
np.sqrt(mse)

In [ ]:
y_pred = loaded_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
np.sqrt(mse)
accuracy_score(y_test, y_pred)